In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load Dataset

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=256, shuffle=True, drop_last = True)

testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=256, shuffle=False, drop_last = True)

Files already downloaded and verified
Files already downloaded and verified


# Model

In [4]:
class VGG(nn.Module):
    def __init__(self):
        super(VGG, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
        )

        self.classifier = nn.Sequential(
            nn.Linear(4 * 4 * 256, 4096),
            nn.Dropout(0.2),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 4096),
            nn.Dropout(0.2),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 512),
            nn.Dropout(0.2),
            nn.ReLU(inplace=True),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(-1, 4 * 4 * 256)
        x = self.classifier(x)
        return x

# Train Function

In [5]:
def train(criterion, optimizer, model, num_epochs, device):
    loss_history = []
    test_acc_history = []

    for epoch in range(num_epochs):
        loss_per_epoch = 0
        for images, label in trainloader:
            optimizer.zero_grad()

            images, label = images.to(device), label.to(device)

            outputs = model(images)

            loss = criterion(outputs, label)
            loss.backward()
            optimizer.step()

            loss_per_epoch += loss

        correct = 0
        total = 0

        with torch.no_grad():
            for images, label in testloader:
                images, label = images.to(DEVICE), label.to(DEVICE)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += label.size(0)
                correct += (predicted == label).sum().item()

        print(f"epoch: {epoch}, loss_per_epoch: {loss_per_epoch}, test_accuracy: {correct/total}")
        loss_history.append(loss_per_epoch.cpu().detach().numpy())
        test_acc_history.append((correct * 100/total))

    return loss_history, test_acc_history

In [6]:
model = VGG()
model = model.to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

num_epochs = 20
loss_history, test_acc_history = train(criterion, optimizer, model, num_epochs = num_epochs, device = DEVICE)

epoch: 0, loss_per_epoch: 341.1842041015625, test_accuracy: 0.4794671474358974
epoch: 1, loss_per_epoch: 240.4415283203125, test_accuracy: 0.5974559294871795
epoch: 2, loss_per_epoch: 188.27964782714844, test_accuracy: 0.6911057692307693
epoch: 3, loss_per_epoch: 149.82054138183594, test_accuracy: 0.7182491987179487
epoch: 4, loss_per_epoch: 121.84396362304688, test_accuracy: 0.7571113782051282
epoch: 5, loss_per_epoch: 97.75029754638672, test_accuracy: 0.7674278846153846
epoch: 6, loss_per_epoch: 78.45774841308594, test_accuracy: 0.7838541666666666
epoch: 7, loss_per_epoch: 60.614967346191406, test_accuracy: 0.7901642628205128
epoch: 8, loss_per_epoch: 43.391990661621094, test_accuracy: 0.7850560897435898
epoch: 9, loss_per_epoch: 32.66335678100586, test_accuracy: 0.7743389423076923
epoch: 10, loss_per_epoch: 24.540250778198242, test_accuracy: 0.7809495192307693
epoch: 11, loss_per_epoch: 17.794069290161133, test_accuracy: 0.7899639423076923
epoch: 12, loss_per_epoch: 16.3006992340087